In [7]:
import re
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

In [12]:
df1 = pd.read_csv('training.1600000.processed.noemoticon.csv', delimiter=',', encoding='ISO-8859-1')
#df1 = pd.read_csv('/kaggle/input/sentiment140/training.1600000.processed.noemoticon.csv', delimiter=',', encoding='ISO-8859-1')

In [13]:
# Get the first 100 rows
first_100 = df1.head(100)

# Get the last 100 rows
last_100 = df1.tail(100)

# Concatenate both DataFrames
df_first_last_200 = pd.concat([first_100, last_100], ignore_index=True)

# Display the concatenated DataFrame
print(df_first_last_200)

     0  1467810369  Mon Apr 06 22:19:45 PDT 2009  NO_QUERY  _TheSpecialOne_  \
0    0  1467810672  Mon Apr 06 22:19:49 PDT 2009  NO_QUERY    scotthamilton   
1    0  1467810917  Mon Apr 06 22:19:53 PDT 2009  NO_QUERY         mattycus   
2    0  1467811184  Mon Apr 06 22:19:57 PDT 2009  NO_QUERY          ElleCTF   
3    0  1467811193  Mon Apr 06 22:19:57 PDT 2009  NO_QUERY           Karoli   
4    0  1467811372  Mon Apr 06 22:20:00 PDT 2009  NO_QUERY         joy_wolf   
..  ..         ...                           ...       ...              ...   
195  4  2193601966  Tue Jun 16 08:40:49 PDT 2009  NO_QUERY  AmandaMarie1028   
196  4  2193601969  Tue Jun 16 08:40:49 PDT 2009  NO_QUERY      TheWDBoards   
197  4  2193601991  Tue Jun 16 08:40:49 PDT 2009  NO_QUERY           bpbabe   
198  4  2193602064  Tue Jun 16 08:40:49 PDT 2009  NO_QUERY     tinydiamondz   
199  4  2193602129  Tue Jun 16 08:40:50 PDT 2009  NO_QUERY   RyanTrevMorris   

    @switchfoot http://twitpic.com/2y1zl - Awww, th

In [14]:
#Change colume name and value of semtiment
#Value 0 of sentiment is negative
#Value 4(default) of sentiment is positive, change from value of 4 to 1
df_first_last_200.columns = ["sentiment", "id", "date", "query", "user", "text"]
df_first_last_200.replace({'sentiment': {4:1}}, inplace=True)
df1.columns = ["sentiment", "id", "date", "query", "user", "text"]
df1.replace({'sentiment': {4:1}}, inplace=True)

In [15]:
df1

,sentiment,id,date,query,user,text
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew
...,...,...,...,...,...,...
1599994,1,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599995,1,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599996,1,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599997,1,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...


In [16]:
df_first_last_200

,sentiment,id,date,query,user,text
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew
...,...,...,...,...,...,...
195,1,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
196,1,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
197,1,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
198,1,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...


In [17]:
def cleanTxt(text):
    text=re.sub(r'@[A-Za-z0-9]+', '',text) #remove @mention
    text=re.sub(r'#', '', text) #remove the # symbol
    text=re.sub(r'http\S+|www\.\S+', '', text) #remove urls
    text=re.sub(r'\s+', ' ', text).strip() #remove exceesive whitespace
    text=re.sub(r'[^A-Za-z0-9\s]', '', text) #character + number

    return text
    
df1["cleaned_text"] = df1["text"].apply(cleanTxt)
df_first_last_200["cleaned_text"] = df_first_last_200["text"].apply(cleanTxt)

In [18]:
df1

,sentiment,id,date,query,user,text,cleaned_text
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,is upset that he cant update his Facebook by t...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,I dived many times for the ball Managed to sav...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",no its not behaving at all im mad why am i her...
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew,not the whole crew
...,...,...,...,...,...,...,...
1599994,1,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...,Just woke up Having no school is the best feel...
1599995,1,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...,TheWDBcom Very cool to hear old Walt interviews
1599996,1,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...,Are you ready for your MoJo Makeover Ask me fo...
1599997,1,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...,Happy 38th Birthday to my boo of alll time Tup...


In [19]:
#Create dataframe ,preparing for training a model
df2 = pd.DataFrame()
df3 = pd.DataFrame()
df2["cleaned_text"] = df1["cleaned_text"]
df3["sentiment"]=df1["sentiment"]

df_t_2 = pd.DataFrame()
df_t_3 = pd.DataFrame()
df_t_2["cleaned_text"] = df_first_last_200["cleaned_text"]
df_t_3["sentiment"] = df_first_last_200["sentiment"]

In [20]:
df2

,cleaned_text
0,is upset that he cant update his Facebook by t...
1,I dived many times for the ball Managed to sav...
2,my whole body feels itchy and like its on fire
3,no its not behaving at all im mad why am i her...
4,not the whole crew
...,...
1599994,Just woke up Having no school is the best feel...
1599995,TheWDBcom Very cool to hear old Walt interviews
1599996,Are you ready for your MoJo Makeover Ask me fo...
1599997,Happy 38th Birthday to my boo of alll time Tup...


In [21]:
df3

,sentiment
0,0
1,0
2,0
3,0
4,0
...,...
1599994,1
1599995,1
1599996,1
1599997,1


In [23]:
#Inserting value each value in x and y
X = df2["cleaned_text"]    # Features
y = df3["sentiment"]   # sentiment score

X_t = df_t_2["cleaned_text"]
y_t = df_t_3["sentiment"]

In [24]:
#All data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [44]:
#Vectorize the text data
tfidf_vectorizer = TfidfVectorizer(max_features=100000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

In [26]:
#Initialize and train the model
model = LogisticRegression(max_iter=1000)
model.fit(X_train_tfidf, y_train)

LogisticRegression(max_iter=1000)

In [45]:
#Predict and evaluate
y_pred = model.predict(X_test_tfidf)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.799740625
Classification Report:
               precision    recall  f1-score   support

           0       0.81      0.79      0.80    159494
           1       0.79      0.81      0.80    160506

    accuracy                           0.80    320000
   macro avg       0.80      0.80      0.80    320000
weighted avg       0.80      0.80      0.80    320000



In [57]:
#200 data
X_t_train, X_t_test, y_t_train, y_t_test = train_test_split(X_t, y_t, test_size=0.2, random_state=42)

test_tfidf_vectorizer = TfidfVectorizer(max_features=1000)
test_X_train_tfidf = test_tfidf_vectorizer.fit_transform(X_t_train)
test_X_t_test_tfidf = test_tfidf_vectorizer.transform(X_t_test)

test_model = LogisticRegression(max_iter=1000)
test_model.fit(test_X_train_tfidf, y_t_train)

y_t_pred = test_model.predict(test_X_t_test_tfidf)
t_accuracy = accuracy_score(y_t_test, y_t_pred)
print("Accuracy:", t_accuracy)
print("Classification Report:\n", classification_report(y_t_test, y_t_pred))

Accuracy: 0.575
Classification Report:
               precision    recall  f1-score   support

           0       0.59      0.62      0.60        21
           1       0.56      0.53      0.54        19

    accuracy                           0.57        40
   macro avg       0.57      0.57      0.57        40
weighted avg       0.57      0.57      0.57        40



In [99]:
#New data (example)
new_sentences = [
    "You are so annoying",
    "god bless you",
    "interesting!"
]

In [100]:
new_sentences_tfidf = tfidf_vectorizer.transform(new_sentences)

In [102]:
new_predictions = test_model.predict(new_sentences_tfidf)

In [104]:
for sentence, prediction in zip(new_sentences, new_predictions):
    print(f"Sentence: {sentence}")
    print(f"Predicted label: {prediction}")
    print("------")

Sentence: You are so annoying
Predicted label: 0
------
Sentence: god bless you
Predicted label: 1
------
Sentence: interesting!
Predicted label: 1
------
